In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, random

In [2]:
million=1000000
nbMillions=100
nSamples=50
lexiquePrefix="MGC-160104"
tiragePrefix="MGC-170507"
echantillon="-%dMo"%nbMillions

In [3]:
with open("/Users/gilles/Box Sync/2015-Data/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    lexiqueBase = pickle.load(input)

In [ ]:
lexiqueBase["freqcum"]= (lexiqueBase["freq"].cumsum()*1000).astype(int)

In [ ]:
rangeTop=lexiqueBase["freqcum"].max()

In [ ]:
del lexiqueBase["ext"]
del lexiqueBase["cs"]
del lexiqueBase["ms"]
del lexiqueBase["vs"]
del lexiqueBase["prob"]

In [ ]:
lexiqueBase["tir1"]=0

In [ ]:
def tirageIndex():
    tirage=[]
    for n in range(nbMillions*million):
        tirage.append(random.randrange(rangeTop))
    return tirage

In [ ]:
%%time
def tirage2triage(tirage):    
    triage=sorted(tirage)
    freqTop=0
    indexMin=0
    tirs={}
    lexique=lexiqueBase.copy()
    lexique["tir1"]=0
    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir][0:1].index.astype(int)[0]
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%5000000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]
    return lexique

In [ ]:
for nSample in range(nSamples):
    tirage=tirageIndex()
    %time lexique=tirage2triage(tirage)
    print
    print "nSample",nSample
    print lexique[lexique["tir1"]!=0]["tir1"].sum()
    print lexique[lexique["tir1"]!=0]["tir1"].count()
    with open("/Users/gilles/Box Sync/2015-Data/"+tiragePrefix+"-%2d"%nSample+echantillon+'-Tirage.pkl', 'wb') as output:
        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
#    lexiqueBase["tir1"]+=lexique["tir1"]
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].sum()
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].count()
#with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+tiragePrefix+"-%2d"%nSamples+"-Total"+echantillon+'-Tirage.pkl', 'wb') as output:
#    pickle.dump(lexiqueBase, output, pickle.HIGHEST_PROTOCOL)
